In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles", sql, index_col="id", parse_dates=["datePublished"])

In [ ]:
len(df)

In [ ]:
len(df[df["title"].map(str) != ''])

In [ ]:
df["title_length"] = [len(t.split(" ")) for t in df["title"]]
df["title_length"].describe()

In [ ]:
len(df[df["header"].map(str) == ''])

In [ ]:
df["title_header_length"] = [len(t.split(" ")) for t in (df["title"]+" "+df["header"])]
df["title_header_length"].describe()

In [ ]:
df["no_tokens"].describe()

In [ ]:
paragraphs = [p for text in df["text"] for p in text.split("\n")]

In [ ]:
len(paragraphs)